In [1]:
import numpy as np
import pandas as pd
import numpy.ma as ma
from scipy.optimize import curve_fit
import numpy.linalg as LA
import math
import struct

#modulef = '/home/ubuntu/source/diff-classifier/diff_classifier/'
modulef = 'C:/Users/koolk/Desktop/diff-classifier/diff_classifier/'
import sys
sys.path.insert(0, modulef)

from utils import csv_to_pd
from msd import nth_diff, msd_calc, all_msds

In [3]:
frames = 2000
drift = 0
d = {'Frame': np.linspace(1, frames, frames),
     'Track_ID': np.ones(frames),
             'X': np.random.rand(frames)+drift*np.linspace(0, 1, frames),
             'Y': np.random.rand(frames)+drift*np.linspace(0, 1, frames)}
df = pd.DataFrame(data=d)
df = all_msds(df)
#df

In [34]:
def calculate_features(df):

    # Skeleton of Trajectory features metadata table.
    # Builds entry for each unique Track ID.
    die = {'Track_ID': df.Track_ID.unique(),
          'alpha': df.Track_ID.unique(),
          'D_fit': df.Track_ID.unique(),
          'kurtosis': df.Track_ID.unique(),
          'asymmetry1': df.Track_ID.unique(),
          'asymmetry2': df.Track_ID.unique(),
          'asymmetry3': df.Track_ID.unique(),
          'AR': df.Track_ID.unique(),
          'elongation': df.Track_ID.unique(),
          'boundedness': df.Track_ID.unique(),
          'fractal_dim': df.Track_ID.unique(),
          'trappedness': df.Track_ID.unique(),
          'efficiency': df.Track_ID.unique(),
          'straightness': df.Track_ID.unique(),
          'MSD_ratio': df.Track_ID.unique()}
    di = pd.DataFrame(data=die)

    trackids = df.Track_ID.unique()
    partcount = trackids.shape[0]


    for particle in range(0, partcount):
        single_track = df.loc[df['Track_ID'] == trackids[particle]].sort_values(['Track_ID', 'Frame'],
                                                                                 ascending=[1, 1]).reset_index(drop=True)
        di['alpha'], di['D_fit'] = alpha_calc(single_track)
        di['kurtosis'] = kurtosis(single_track)
        l1, l2, di['asymmetry1'], di['asymmetry2'], di['asymmetry3'] = asymmetry(single_track)
        di['AR'], di['elongation'] = aspectratio(single_track)
        di['boundedness'], di['fractal_dim'], di['trappedness'] = boundedness(single_track)
        di['efficiency'], di['straightness'] = efficiency(single_track)
        di['MSD_ratio'] = msd_ratio(single_track, 2, single_track['Frame'][single_track.shape[0]-2])
    
    return di

In [35]:
calculate_features(df)

,AR,D_fit,MSD_ratio,Track_ID,alpha,asymmetry1,asymmetry2,asymmetry3,boundedness,efficiency,elongation,fractal_dim,kurtosis,straightness,trappedness
0,1.001097,0.083996,3.312166,1.0,5.395747e-09,0.999638,0.009516,0.656478,0.054464,0.000156,0.001096,7.792186,2.429733,0.00031,-0.210605


In [31]:
di

,AR,D_fit,MSD_ratio,Track_ID,alpha,asymmetry1,asymmetry2,asymmetry3,boundedness,efficiency,elongation,fractal_dim,kurtosis,straightness,trappedness
0,1.001097,0.083996,3.312166,1.0,5.395747e-09,0.999638,0.009516,0.656478,0.054464,0.000156,0.001096,7.792186,2.429733,0.00031,-0.210605


In [9]:
def alpha_calc(track):
    """
    Calculates the parameter alpha by fitting track MSD data to a function.
    
    Parameters
    ----------
    
    Returns
    ----------
    
    Examples
    ----------
    """

    x = track['Frame'] - 1
    y = track['MSDs']

    def msd_alpha(x, a, D):
        return 4*D*(x**a)

    try:
        popt, pcov = curve_fit(msd_alpha, x, y)
        a = popt[0]
        D = popt[1]
    except RuntimeError:
        print('Optimal parameters not found. Print NaN instead.')
        a = np.nan
        D = np.nan
    return a, D

In [10]:
def gyration_tensor(track):
    
    df = track
    
    Ta = np.sum((df['X'] - np.mean(df['X']))**2)/df['X'].shape[0]
    Tb = np.sum((df['Y'] - np.mean(df['Y']))**2)/df['Y'].shape[0]
    Tab = np.sum((df['X'] - np.mean(df['X']))*(df['X'] - np.mean(df['X'])))/df['X'].shape[0]

    w, v = LA.eig(np.array([[Ta, Tab], [Tab, Tb]]))
    dom = np.argmax(np.abs(w))
    rec = np.argmin(np.abs(w))
    l1 = w[dom]
    l2 = w[rec]
    v1 = v[dom]
    v2 = v[rec]
    return l1, l2, v1, v2

In [11]:
def kurtosis(track):
    df = track
    l1, l2, v1, v2 = gyration_tensor(df)
    projection = df['X']*v1[0] + df['Y']*v1[1]

    kurt = np.mean((projection - np.mean(projection))**4/(np.std(projection)**4))
    
    return kurt

In [14]:
def asymmetry(track):

    l1, l2, v1, v2 = gyration_tensor(track)
    a1 = (l1**2 - l2**2)**2/(l1**2 + l2**2)**2
    a2 = l2/l1
    a3 = -np.log(1-((l1-l2)**2)/(2*(l1+l2)**2))

    return l1, l2, a1, a2, a3

In [15]:
l1, l2, a1, a2, a3 = asymmetry(df)

In [16]:
def aspectratio(track):
    rot_angle, area, width, height, center_point, corner_points = minBoundingRect(df)
    ar = width/height
    if ar > 1:
        counter = 1
    else:
        ar = 1/ar
    elong = 1 - (1/ar)

    return ar, elong

In [ ]:
aspectratio(df)

In [17]:
def minBoundingRect(df):
    
    #Based off of code from the following repo:
    # https://github.com/dbworth/minimum-area-bounding-rectangle/blob/master/python/min_bounding_rect.py

    df2 = np.zeros((df.shape[0]+1, 2))
    df2[:-1, :] = df[['X', 'Y']].values
    df2[-1, :] = df[['X', 'Y']].values[0, :]
    hull_points_2d = df2

    edges = np.zeros((len(hull_points_2d)-1, 2))

    for i in range(len(edges)):
        edge_x = hull_points_2d[i+1,0] - hull_points_2d[i,0]
        edge_y = hull_points_2d[i+1,1] - hull_points_2d[i,1]
        edges[i] = [edge_x,edge_y]

    edge_angles = np.zeros((len(edges)))

    for i in range(len(edge_angles)):
        edge_angles[i] = math.atan2( edges[i,1], edges[i,0] )
    edge_angles = np.unique(edge_angles)

    start_area = platform_c_maxint = 2 ** (struct.Struct('i').size * 8 - 1) - 1
    min_bbox = (0, start_area, 0, 0, 0, 0, 0, 0)
    for i in range( len(edge_angles) ):
        R = np.array([ [ math.cos(edge_angles[i]), math.cos(edge_angles[i]-(math.pi/2)) ],
                     [ math.cos(edge_angles[i]+(math.pi/2)), math.cos(edge_angles[i]) ] ])

        rot_points = np.dot(R, np.transpose(hull_points_2d) )

        min_x = np.nanmin(rot_points[0], axis=0)
        max_x = np.nanmax(rot_points[0], axis=0)
        min_y = np.nanmin(rot_points[1], axis=0)
        max_y = np.nanmax(rot_points[1], axis=0)

        width = max_x - min_x
        height = max_y - min_y
        area = width*height

        if (area < min_bbox[1]):
            min_bbox = (edge_angles[i], area, width, height, min_x, max_x, min_y, max_y)

    angle = min_bbox[0]   
    R = np.array([ [ math.cos(angle), math.cos(angle-(math.pi/2)) ], [ math.cos(angle+(math.pi/2)), math.cos(angle) ] ])
    proj_points = np.dot(R, np.transpose(hull_points_2d) ) # 2x2 * 2xn

    min_x = min_bbox[4]
    max_x = min_bbox[5]
    min_y = min_bbox[6]
    max_y = min_bbox[7]

    center_x = (min_x + max_x)/2
    center_y = (min_y + max_y)/2
    center_point = np.dot( [ center_x, center_y ], R )

    corner_points = np.zeros( (4,2) )
    corner_points[0] = np.dot( [ max_x, min_y ], R )
    corner_points[1] = np.dot( [ min_x, min_y ], R )
    corner_points[2] = np.dot( [ min_x, max_y ], R )
    corner_points[3] = np.dot( [ max_x, max_y ], R )

    return (angle, min_bbox[1], min_bbox[2], min_bbox[3], center_point, corner_points)
    # rot_angle, area, width, height, center_point, corner_points

In [18]:
def boundedness(track, framerate = 1):

    df = track
    length = df.shape[0]
    distance = np.zeros((length, length))

    for frame in range(0, length-1):
        distance[frame, 0:length-frame-1] = (np.sqrt(nth_diff(df['X'], frame+1)**2 + nth_diff(df['Y'], frame+1)**2).values)
    
    L = np.sum((np.sqrt(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2).values))
    r = np.max(distance)/2
    f = (length-1)*framerate
    D = df['MSDs'][length-1]/(4*f)
    
    B = D*f/(r**2)
    Df = np.log(length-1)/np.log((length-1)*2*r/L)
    pf = 1 - np.exp(0.2048 - 0.25117*(D*f/(r**2)))

    return B, Df, pf

In [19]:
def efficiency(track):

    df = track
    length = df.shape[0]
    num = (nth_diff(df['X'], length-1)**2 + nth_diff(df['Y'], length-1)**2)[0]
    num2 = np.sqrt(num)
    
    den = np.sum(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2)
    den2 = np.sum(np.sqrt(nth_diff(df['X'], 1)**2 + nth_diff(df['Y'], 1)**2))
    
    eff = num/den
    strait = num2/den2
    return eff, strait

In [20]:
def msd_ratio(track, n1=3, n2=100):

    df = track
    assert n1 < n2, "n1 must be less than n2"
    ratio = (df['MSDs'][n1]/df['MSDs'][n2]) - (df['Frame'][n1]/df['Frame'][n2])
    return ratio

In [ ]:
msd_ratio(df, 1, 100)